In [1]:
#-----------Importing Modules--------------
# Sign-in
from pydrive.auth import GoogleAuth
# Creating Service 
from pydrive.drive import GoogleDrive
# To work with time
import datetime
from datetime import date
# Import os to work with local files and folders, examining user OS 
from os import listdir
from os.path import isfile, join
import os.path
# Working with mail
import smtplib, ssl

In [ ]:
#-----------Sign-In Process----------------
# Automating the Process
gauth = GoogleAuth()

In [ ]:
# Try to load saved client credentials
gauth.LoadCredentialsFile("mycreds.txt")

if gauth.credentials is None:
    # Authenticate if they're not there

    # This is what solved the issues:
    gauth.GetFlow()
    gauth.flow.params.update({'access_type': 'offline'})
    gauth.flow.params.update({'approval_prompt': 'force'})

    gauth.LocalWebserverAuth()

elif gauth.access_token_expired:

    # Refresh them if expired

    gauth.Refresh()
else:

    # Initialize the saved creds

    gauth.Authorize()

# Save the current credentials to a file
gauth.SaveCredentialsFile("mycreds.txt")  


In [ ]:
#Initiating the Service
drive = GoogleDrive(gauth)

In [ ]:
#Getting all folders data such as "Name" and "ID"
def GetAllFolder():
    folders = drive.ListFile({'q': "mimeType = 'application/vnd.google-apps.folder' and trashed = false"}).GetList()
    return folders

In [ ]:
#Creating a table for all folders
def ShowAllFolders(folders):
    x = 1
    print("Here are all of your Folders")
    getMaxTitleLen = 0
    getMaxIdLen = 0
    for folder in folders : # Return the longest Name and ID of the table 
        getMaxTitleLen = max(getMaxTitleLen, len(folder['title']))
        getMaxIdLen = max(getMaxIdLen, len(folder['id']))
    # Print folders table
    numColLen = len('No.') + len(' ' * 6)
    titColLen = getMaxTitleLen + 6
    idColLen = getMaxIdLen + 6
    # Print the first line of the table with No, Title, and ID
    print('No.' + ' ' * (numColLen - len('No.')) + '|TITLE' + ' ' * (titColLen - len('TITLE')) + '|ID')
    print('=' * (numColLen + titColLen + idColLen)) #seperated by ===================================
    No = 2
    print(str(x) + ' ' * (numColLen - len(str(x))) + '|' + 'root' + ' ' * (titColLen - len('root')) + '|' + 'root')
    print('-' * (numColLen + titColLen + idColLen))
    for folder in folders:
        print(str(No) + ' ' * (numColLen - len(str(No))) + '|' + folder['title'] + ' ' * (titColLen - len(folder['title'])) + '|' + folder['id'])
        print('-' * (numColLen + titColLen + idColLen))
        No += 1

In [ ]:
def CreateANewFolder(parent_name, folder_name):
    folders = GetAllFolder()
    parent_id = ""
    for folder in folders : 
        if folder['title'] == parent_name : # Creating folder 
            parent_id = folder['id']
            break
    if parent_name == 'root' :# Uploading directly to root folder
        newFolder = drive.CreateFile({
            'title': '[' + str(date.today()) + '] ' + folder_name,
            'mimeType': 'application/vnd.google-apps.folder'
        })
    else : # Creating a sub-folder
        newFolder = drive.CreateFile({
            'title': '[' + str(date.today()) + '] ' + folder_name,
            'parents': [{'id': parent_id}],
            'mimeType': 'application/vnd.google-apps.folder'
        })
    newFolder.Upload()
    return newFolder

In [ ]:
def file_path(filesContainer): #fixing issues with lacking '/' or '\\' at the end of the path
    import platform
    path = ''
    if platform.system() == 'Linux': #linux directory will be ABC/filenames/
        if list(filesContainer)[-1:] == '/':
            path = filesContainer
        else:
            path = str(filesContainer)+str('/')
    elif platform.system() == 'Windows':
        if list(filesContainer)[-2:] == '\\':
            path = filesContainer
        else:
            path = str(filesContainer)+str('\\')
        
    return path 

In [ ]:
def UploadFiles(path,folderId) :
    drive = GoogleDrive(gauth)
    numFiles = 0
    detailInfo = ""
    for file in listdir(filesContainer) :
        if folderId.lower() != 'root' :
            uploadFile = drive.CreateFile({
                'parents' : [{'id' : folderId}],
                'title': file[ : -4]
            })
        else :
            uploadFile = drive.CreateFile({
                'title': file[ : -4]
            })
        uploadFile.SetContentFile(filesContainer + '/' + file)
        uploadFile.Upload()
        numFiles += 1
        detailInfo += """
        %d. %s
            Size: %s
            Modified Date: %s
        - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
        """ % (numFiles, uploadFile['title'], str(round(int(uploadFile['fileSize'])/1024, 2)) + ' KB', uploadFile['modifiedDate'][ : 10])
    print('\n' + str(numFiles) + ' file(s) successfully uploaded')
    return (numFiles, detailInfo)

In [ ]:
from PyQt5 import QtCore, QtGui, QtWidgets

class Ui_MainWindow(object):
    def setupUi(self, MainWindow):
        MainWindow.setObjectName("MainWindow")
        MainWindow.resize(250, 346)
        self.centralwidget = QtWidgets.QWidget(MainWindow)
        self.centralwidget.setObjectName("centralwidget")
        self.ShowAllFolders = QtWidgets.QPushButton(self.centralwidget)
        self.ShowAllFolders.setGeometry(QtCore.QRect(-10, 0, 261, 271))
        self.ShowAllFolders.setObjectName("ShowAllFolders")
        self.progressBar = QtWidgets.QProgressBar(self.centralwidget)
        self.progressBar.setGeometry(QtCore.QRect(0, 290, 118, 23))
        self.progressBar.setProperty("value", 24)
        self.progressBar.setObjectName("progressBar")
        MainWindow.setCentralWidget(self.centralwidget)
        self.toolBar = QtWidgets.QToolBar(MainWindow)
        self.toolBar.setObjectName("toolBar")
        MainWindow.addToolBar(QtCore.Qt.TopToolBarArea, self.toolBar)
        self.toolBar_2 = QtWidgets.QToolBar(MainWindow)
        self.toolBar_2.setObjectName("toolBar_2")
        MainWindow.addToolBar(QtCore.Qt.TopToolBarArea, self.toolBar_2)
        self.menubar = QtWidgets.QMenuBar(MainWindow)
        self.menubar.setGeometry(QtCore.QRect(0, 0, 250, 22))
        self.menubar.setObjectName("menubar")
        self.menuFile = QtWidgets.QMenu(self.menubar)
        self.menuFile.setObjectName("menuFile")
        self.menuHelp = QtWidgets.QMenu(self.menubar)
        self.menuHelp.setObjectName("menuHelp")
        MainWindow.setMenuBar(self.menubar)
        self.actionYour_Email = QtWidgets.QAction(MainWindow)
        self.actionYour_Email.setObjectName("actionYour_Email")
        self.actionClose = QtWidgets.QAction(MainWindow)
        self.actionClose.setObjectName("actionClose")
        self.actionCredits = QtWidgets.QAction(MainWindow)
        self.actionCredits.setObjectName("actionCredits")
        self.actionOpen_Docs = QtWidgets.QAction(MainWindow)
        self.actionOpen_Docs.setObjectName("actionOpen_Docs")
        self.menuFile.addAction(self.actionYour_Email)
        self.menuFile.addAction(self.actionClose)
        self.menuHelp.addAction(self.actionCredits)
        self.menuHelp.addAction(self.actionOpen_Docs)
        self.menubar.addAction(self.menuFile.menuAction())
        self.menubar.addAction(self.menuHelp.menuAction())

        self.retranslateUi(MainWindow)
        QtCore.QMetaObject.connectSlotsByName(MainWindow)

    def retranslateUi(self, MainWindow):
        _translate = QtCore.QCoreApplication.translate
        MainWindow.setWindowTitle(_translate("MainWindow", "MainWindow"))
        self.ShowAllFolders.setStatusTip(_translate("MainWindow", "Showing "))
        self.ShowAllFolders.setText(_translate("MainWindow", "Start "))
        self.toolBar.setWindowTitle(_translate("MainWindow", "toolBar"))
        self.toolBar_2.setWindowTitle(_translate("MainWindow", "toolBar_2"))
        self.menuFile.setTitle(_translate("MainWindow", "File"))
        self.menuHelp.setTitle(_translate("MainWindow", "Help"))
        self.actionYour_Email.setText(_translate("MainWindow", "Your Email"))
        self.actionClose.setText(_translate("MainWindow", "Close"))
        self.actionCredits.setText(_translate("MainWindow", "Credits"))
        self.actionOpen_Docs.setText(_translate("MainWindow", "Open Docs"))

In [ ]:
# ===============================================
# = = = = = = = = = MAIN DRIVER = = = = = = = = =
# ===============================================

import getpass    # https://www.geeksforgeeks.org/getpass-and-getuser-in-python-password-without-echo/

# Get all user's drive folders
folders = GetAllFolder()

# Display them under a table
ShowAllFolders(folders)

# Ask user for his/her option
folderId = ""
option = input('Do you want to upload files to existing folders or create a new one?\n1. Upload to existing folders\n2. Create a new folder\n')
if(int(option) == 1) :
    # Ask user for their desire drive folder to upload files
    folderId = input('Please specify the folder id you want to upload files: > ')

else :
    try:
        parent_name = input('Please enter a parent folder name: ')
        folder_name = input('Please enter name of a new folder: ')
        newFolder = CreateANewFolder(parent_name, folder_name)
        folderId = newFolder['id']
    except Exception as e:
        print("Please enter ")
    

# Ask user for their files container on local PC
filesContainer = input('Please enter the files path: > ')# C:\\Users\\dacluc\\Desktop\\Upload_Files
path = file_path(filesContainer)

# Ask user for their email address to inform them when the uploading process finished
# receiverEmail = getpass.getpass(prompt = "Please enter your email address, we'll inform you when uploading completed: > ")
receiverEmail = input("Please enter your email address, we'll inform you when uploading completed: > ")

# Files uploading starts from now . . .
try:
    numFiles, detailInfo = UploadFiles(filesContainer, folderId)
except FileNotFoundError:
    print("No such file or directory. Please try again")
except Exception as e:
    print("Some errors had occured during the process. Please try again")

# - - - - - - SEND MAIL TO USER - - - - - -

port = 587  # For starttls
smtp_server = "smtp.gmail.com"
sender_email = "phonglearntodev@gmail.com"    # Your email address
receiver_email = receiverEmail
# password = input("Type your password and press enter:")
password = 'Phong2307hn'    # Your password
message = """\
Subject: Noti from Google Drive

Hooray! %d files(s) have been successfully uploaded
- - - - - - - - -

%s
""" % (numFiles, detailInfo)
context = ssl.create_default_context()
with smtplib.SMTP(smtp_server, port) as server:
#     server.ehlo()  # Can be omitted
    server.starttls(context=context)
#     server.ehlo()  # Can be omitted
    server.login(sender_email, password)
    server.sendmail(sender_email, receiver_email, message)